In [1]:
import numpy as np
import pandas as pd

In [2]:
from bs4 import BeautifulSoup
import requests
import time, os, io

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
from PIL import Image, ImageFilter
import hashlib

In [5]:
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException

In [6]:
chromedriver = "/Users/chromedriver" # path to the chromedriver
os.environ["webdriver.chrome.driver"] = chromedriver

In [ ]:
def find_photos(link, num_scrolls, type_clothing):
    driver = webdriver.Chrome(chromedriver)
    driver.get(link)
    
    time.sleep(2)
    #to remove banner   
    close_banner = driver.find_element_by_xpath('//button[contains(@class, "banner-wrapper__close-icon")]')
    close_banner.click()
    time.sleep(3)
    #to remove pop-up ad to continue
    #close_button = driver.find_element_by_xpath('//button[contains(@class, "close ng-star-inserted")]')
    #close_button.click()
    #time.sleep(2)
    
    for i in range(num_scrolls):
        #Scroll
        driver.execute_script(
            "window.scrollTo(0, document.documentElement.scrollHeight);" #Alternatively, document.body.scrollHeight
        )

        #Wait for page to load
        time.sleep(3)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    products = soup.find_all('div', class_="product-cell u-text-center product-cell--desktop")
    
    prod_dic = {}
    for i,product in enumerate(products):
        
        #img link
        link_img = product.find_all('link')[1].get('href')
        #price
        price = product.find('span', class_='product-cell__price').text.strip()
        #product link
        web_link = product.find_all('link')[0].get('href')
        #product name
        product_name = product.find(class_='product-cell__label u-block u-capitalize').text.strip()
        #type
        product_type = type_clothing
        prod_dic[i] = [product_name, product_type, web_link, price, link_img]
    
    driver.quit()
    
    return prod_dic

In [ ]:
def image_scrape(url,folder_path,type_product):
    if url.startswith('/'):
        return np.nan
    else:
        try:
            image_content = requests.get(url).content

        except Exception as e:
            print(f"ERROR - Could not download {url} - {e}")

        try:
            image_file = io.BytesIO(image_content)
            image = Image.open(image_file).convert('RGB')
            img_name = type_product + '_' + hashlib.sha1(image_content).hexdigest()[:10] + '.jpg'
            file_path = os.path.join(folder_path,img_name)
            with open(file_path, 'wb') as f:
                image.save(f, "JPEG", quality=85)
            #print(f"SUCCESS - saved {url} - as {file_path}")
        except Exception as e:
            print(f"ERROR - Could not save {url} - {e}")
        return img_name

### **Scraping images by categories**

**Dresses**

In [8]:
url1 = 'https://www.shopstyle.com/browse/day-dresses?sort=Popular'

In [9]:
dres_new = find_photos(url, 200, 'dress')

In [10]:
df_dres_new = pd.DataFrame(dres_new).T

In [53]:
df_dres_new['img'] = df_dres_new[4]

In [54]:
folder_dres = '/Users/..../dresses' # path to where images will be saved

In [63]:
df_dres_new['img'] = df_dres_new['img'].apply(image_scrape, folder_path=folder_dres, type_product='dress')

In [67]:
df_dres_new.columns = ['product_name', 'product_type', 'web_link', 'price', 'link_img', 'img_name']

In [70]:
#pickle!
df_dres_new.to_pickle("./df_dres_new.pkl")

**Denim**

In [23]:
url2 = 'https://www.shopstyle.com/browse/jeans?sort=Popular'

In [24]:
denim = find_photos(url2, 200, 'denim')

In [25]:
df_denim = pd.DataFrame(denim).T

In [26]:
df_denim.shape

(5960, 5)

In [27]:
df_denim['img'] = df_denim[4]

In [40]:
df_denim.head(1)

,0,1,2,3,4,img
0,Gloria Vanderbilt Women's Comfort Curvy Bootc...,denim,https://www.shopstyle.com/g/women/gloria-vande...,$29.99,https://img.shopstyle-cdn.com/sim/cb/8d/cb8d59...,https://img.shopstyle-cdn.com/sim/cb/8d/cb8d59...


In [41]:
folder_denim = '/Users/..../denim' 

In [42]:
df_denim['img'] = df_denim['img'].apply(image_scrape, folder_path=folder_denim, type_product='denim')

In [79]:
df_denim.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5960 entries, 0 to 5959
Data columns (total 6 columns):
product_name    5960 non-null object
product_type    5960 non-null object
web_link        5717 non-null object
price           5960 non-null object
link_img        5960 non-null object
img_name        5960 non-null object
dtypes: object(6)
memory usage: 325.9+ KB


In [80]:
df_denim.columns = ['product_name', 'product_type', 'web_link', 'price', 'link_img', 'img_name']

**skirts**

In [71]:
url3 = 'https://www.shopstyle.com/browse/mid-length-skirts?sort=Popular'

In [72]:
skirts_new = find_photos(url6,200,'skirts')

In [73]:
df_skirts_new = pd.DataFrame(skirts_new).T

In [74]:
df_skirts_new.shape

(8040, 5)

In [13]:
folder_skirts = '/Users/..../skirts' 

In [81]:
df_skirts_new['img'] = df_skirts_new[4]

In [14]:
df_skirts_new['img'] = df_skirts_new['img'].apply(image_scrape, folder_path=folder_skirts, type_product='skirts')

In [17]:
df_skirts_new.columns = ['product_name', 'product_type', 'web_link', 'price', 'link_img', 'img_name']

**shorts**

In [101]:
url4 = 'https://www.shopstyle.com/browse/shorts'

In [102]:
shorts = find_photos(url7,200,'shorts')

In [103]:
df_shorts = pd.DataFrame(shorts).T

In [104]:
folder_shorts = '/Users/..../shorts' 

In [105]:
df_shorts['img'] = df_shorts[4]

In [106]:
df_shorts.shape

(6720, 6)

In [108]:
df_shorts['img'] = df_shorts['img'].apply(image_scrape, folder_path=folder_shorts, type_product='shorts')

In [109]:
df_shorts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6720 entries, 0 to 6719
Data columns (total 6 columns):
0      6720 non-null object
1      6720 non-null object
2      6227 non-null object
3      6720 non-null object
4      6720 non-null object
img    6718 non-null object
dtypes: object(6)
memory usage: 367.5+ KB


In [110]:
df_shorts.head(1)

,0,1,2,3,4,img
0,Ganni Printed Cotton Poplin Shorts in Egret |...,shorts,https://www.shopstyle.com/g/women/ganni/printe...,$101,https://img.shopstyle-cdn.com/sim/ce/30/ce3080...,shorts_ac2b1d4994.jpg


In [111]:
df_shorts.columns = ['product_name', 'product_type', 'web_link', 'price', 'link_img', 'img_name']

**Tops**

In [ ]:
url5 = 'https://www.shopstyle.com/browse/longsleeve-tops?sort=Popular'

In [ ]:
tops = find_photos(url5,200, 'tops')

In [ ]:
df_tops = pd.DataFrame(tops).T

In [ ]:
folder_tops = '/Users/..../tops' 

In [ ]:
df_tops['img'] = df_tops[4]

In [ ]:
df_tops['img'] = df_tops['img'].apply(image_scrape, folder_path=folder_tops, type_product='tops')

In [ ]:
df_tops.columns = ['product_name', 'product_type', 'web_link', 'price', 'link_img', 'img_name']

**Merging all DataFrames into one**

In [16]:
# ALL Together
df_all_new = pd.concat([df_dres_new, df_skirts_new, df_denim, df_tops, df_shorts])

In [21]:
df_all_new.reset_index(inplace=True)

In [22]:
df_all_new.drop('index', axis=1, inplace=True)

In [25]:
#pickle!
df_all_new.to_pickle("./ ... .pkl")

##df_all_new = pd.read_pickle("./ .. .pkl")

**disclaimer**: _This project has gone through several iterations, the code above is a cleaned-up version. As a result references might vary from other notebooks_ <br>
